In [23]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [24]:
links=pd.read_excel("links.xlsx")
ano=r"\d{4}"

In [25]:
links

,Dados,Links
0,IPCA,/ExibeSerie.aspx?serid=36482&module=M
1,PIB,/ExibeSerie.aspx?serid=521274780
2,INPC,/ExibeSerie.aspx?serid=36472&module=M
3,Dólar,/ExibeSerie.aspx?serid=32098&module=M
4,Salário mínimo vigente,/ExibeSerie.aspx?serid=1739471028&module=M
5,SELIC acumulada,/ExibeSerie.aspx?serid=32241&module=M
6,Desemprego (até 2016.02),/ExibeSerie.aspx?serid=40326&module=M
7,CDI,/ExibeSerie.aspx?serid=32237&module=M


In [46]:
def get_data(dado,link):

    #pega html do site
    html=requests.get(f"http://ipeadata.gov.br{link}").content

    #tratamento com o BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    #filtra o que queremos, que é o que está dentro da tag <td> com a classe "dxgv", a parte string=True garante que não vamos pegar strings vazias
    td_tags = soup.find_all('td',class_="dxgv",string=True)

    #extrai o texto de dentro da tag, que é o dado que queremos mesmo
    tds = [td.text for td in td_tags]
   
    #transforma a lista obtida em um dataframe com duas colunas, e define o nome das colunas
    df=pd.DataFrame([tds[n:n+2] for n in range(0, len(tds), 2)])
    df.columns=["data",dado]
    #corrige pontuação das casas de milhar e da vírgula, além de transformar o tipo do dado para númerico (isso é util para quando formos exportar para csv)
    df[dado]=pd.to_numeric(df[dado].apply(lambda x: str(x).strip().replace(".","").replace(",",".")))
    #separa mês do ano
    df[["ano", "mês"]] = df["data"].str.split(".", expand = True)
    #calcula variação percentual
    df["variação"]= df[dado].pct_change()
    return df

In [47]:
get_data("IPCA","/ExibeSerie.aspx?serid=36482&module=M")

,data,IPCA,ano,mês,variação
0,1979.12,7.618300e-09,1979,12,NaN
1,1980.01,8.122300e-09,1980,01,0.066156
2,1980.02,8.497300e-09,1980,02,0.046169
3,1980.03,9.010400e-09,1980,03,0.060384
4,1980.04,9.486700e-09,1980,04,0.052861
...,...,...,...,...,...
513,2022.09,6.370340e+03,2022,09,-0.002900
514,2022.10,6.407930e+03,2022,10,0.005901
515,2022.11,6.434200e+03,2022,11,0.004100
516,2022.12,6.474090e+03,2022,12,0.006200


In [49]:
dfs={}

In [50]:
for index,row in links.iterrows():
    dado=links.loc[index,"Dados"]
    link=links.loc[index,"Links"]
    df=get_data(dado,link) 
    dfs[dado]=df

In [55]:
dfs['Dólar']

,data,Dólar,ano,mês,variação
0,1953.01,6.683600e-15,1953,01,NaN
1,1953.02,1.337800e-14,1953,02,1.001616
2,1953.03,1.337800e-14,1953,03,0.000000
3,1953.04,1.488400e-14,1953,04,0.112573
4,1953.05,1.427600e-14,1953,05,-0.040849
...,...,...,...,...,...
836,2022.09,5.236400e+00,2022,09,0.018220
837,2022.10,5.249700e+00,2022,10,0.002540
838,2022.11,5.274100e+00,2022,11,0.004648
839,2022.12,5.241800e+00,2022,12,-0.006124
